In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_csv("train.csv", index_col = 0)

In [3]:
df = df.drop(columns=["Candidate"])#Constituency
df = df.drop(columns=["Constituency ∇"])
# print(df.head())

In [4]:
df.replace(" Crore\+", '0000000', inplace=True, regex=True)
df.replace(" Lac\+", '00000', inplace=True, regex=True)
df.replace(" Thou\+", '000', inplace=True, regex=True)
df.replace(" Hund\+", '00', inplace=True, regex=True)
# df.head()

In [6]:
df["Total Assets"]=df["Total Assets"].astype(np.int64)
df["Liabilities"]=df["Liabilities"].astype(np.int64)

In [7]:
# Store the last column as y 
y = df.iloc[:, -1]

# Remove the last column from the original DataFrame
df = df.iloc[:, :-1]
# print(y.head())
# print(X.head())

In [8]:
one_hot_encoded_df = pd.get_dummies(df)
# one_hot_encoded_df.info()

In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
# print(np.unique(y_encoded))

In [ ]:
# To evaluate model performance
# X_train, X_test, y_train, y_test = train_test_split(one_hot_encoded_df, y_encoded,test_size=0.2, random_state = 0) 

In [10]:
# For the submission round
X_train = one_hot_encoded_df
y_train = y_encoded

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(random_state = 42)

param_grid = {
    'max_depth': [1, 2, 3, 5, 7, None],  # Maximum depth of the tree
    'min_samples_split': [2, 4, 6, 8, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4, 6],  # Minimum number of samples required to be at a leaf node
    'criterion': ['gini', 'entropy']  # Split criterion: Gini impurity or information gain
}
grid_search = GridSearchCV(dtree_model, param_grid, cv=5, scoring='f1_weighted', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
# print("Best parameters found: ", grid_search.best_params_)

# Get best model from grid search
best_dtree_model = grid_search.best_estimator_

# test_predictions = best_dtree_model.predict(X_test) 
  
# # creating a confusion matrix 
# cm = confusion_matrix(y_test, test_predictions)

# accuracy = accuracy_score(y_test, test_predictions)
# precision = precision_score(y_test, test_predictions, average='weighted')
# recall = recall_score(y_test, test_predictions, average='weighted')
# f1 = f1_score(y_test, test_predictions, average='weighted')

# # Print the evaluation metrics
# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier()

param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  # Number of neighbors to consider
    'weights': ['uniform', 'distance'],  # Weighting scheme for predictions
    'metric': ['euclidean', 'manhattan']  # Distance metric to use
}

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='f1_weighted', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
# print("Best parameters found: ", grid_search.best_params_)

# Get best model from grid search
best_knn = grid_search.best_estimator_

# test_predictions = best_knn.predict(X_test) 
  
# # creating a confusion matrix 
# cm = confusion_matrix(y_test, test_predictions)

# accuracy = accuracy_score(y_test, test_predictions)
# precision = precision_score(y_test, test_predictions, average='weighted')
# recall = recall_score(y_test, test_predictions, average='weighted')
# f1 = f1_score(y_test, test_predictions, average='weighted')

# # Print the evaluation metrics
# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 300],  
    'max_depth': [None, 3, 5, 7, 10, 15],  
    'min_samples_split': [2, 5, 10],  
    'min_samples_leaf': [1, 2, 4],  
    'max_features': ['log2', 'sqrt',None],  
    'bootstrap': [True, False] 
}
grid_search = GridSearchCV(rf_model, param_grid, cv=5, scoring='f1_weighted', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
# print("Best parameters found: ", grid_search.best_params_)

# Get best model from grid search
best_rf_model = grid_search.best_estimator_

# Train best model on full training data
best_rf_model.fit(X_train, y_train)

# test_predictions = best_rf_model.predict(X_test) 
  
# # creating a confusion matrix 
# cm = confusion_matrix(y_test, test_predictions)

# accuracy = accuracy_score(y_test, test_predictions)
# precision = precision_score(y_test, test_predictions, average='weighted')
# recall = recall_score(y_test, test_predictions, average='weighted')
# f1 = f1_score(y_test, test_predictions, average='weighted')

# # Print the evaluation metrics
# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)

Fitting 5 folds for each of 972 candidates, totalling 4860 fits


RandomForestClassifier(max_features='log2', min_samples_split=5,
                       random_state=42)

In [ ]:
print("Best parameters found: ", grid_search.best_params_)

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

param = {
    'max_depth': 15,  # Maximum depth of the tree
    'eta': 0.3,      # Learning rate
    'objective': 'multi:softmax',  # Specify multiclass classification
    'num_class': 10,
    'eval_metric': 'merror'         # Evaluation metric to monitor
}

# Train the XGBoost model
num_round = 100  # Number of boosting rounds
bst = xgb.train(param, dtrain, num_round)

# Make predictions on the test data
preds = bst.predict(dtest)
test_predictions = preds

cm = confusion_matrix(y_test, test_predictions)

accuracy(y_test, test_predictions)
precision = precision_score(y_test, test_predictions, average='weighted')
recall = recall_score(y_test, test_predictions, average='weighted')
f1_score(y_test, test_predictions, average='weighted')

# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [14]:
def preprocess(file_path):
    test_db = pd.read_csv(file_path, index_col = 0)
    test_db = test_db.drop(columns=["Candidate"])#Constituency
    test_db = test_db.drop(columns=["Constituency ∇"])
    test_db.replace(" Crore\+", '0000000', inplace=True, regex=True)
    test_db.replace(" Lac\+", '00000', inplace=True, regex=True)
    test_db.replace(" Thou\+", '000', inplace=True, regex=True)
    test_db.replace(" Hund\+", '00', inplace=True, regex=True)
    test_db["Total Assets"]=test_db["Total Assets"].astype(np.int64)
    test_db["Liabilities"]=test_db["Liabilities"].astype(np.int64)
    one_hot_encoded_df = pd.get_dummies(test_db)
    return one_hot_encoded_df


In [16]:
test_db = preprocess("test.csv")
test_preds = best_rf_model.predict(test_db)
test_preds = test_preds.astype(int)
converted_data = label_encoder.inverse_transform(test_preds)

In [17]:
submit_df = pd.DataFrame(converted_data)
indices = [i for i in range(len(submit_df))]
submit_df.insert(0, "ID", indices)
submit_df.columns = ["ID","Education"]
submit_df.head()
# len(submit_df)

,ID,Education
0,0,12th Pass
1,1,8th Pass
2,2,Graduate Professional
3,3,Post Graduate
4,4,Graduate


In [18]:
submit_df.to_csv("submit_rf.csv",index = False)